DSCI 100 Project

name: Ryan Cheng 

student ID: 53355756

Question to answer: 

    Question 2: We would like to know which "kinds" of players are most likely to contribute a large amount of data so that we can target those players in our recruiting efforts.

In [7]:
# libraries / plot setting 
library(tidyverse)
library(tidymodels)
library(repr)
library(GGally)
options(repr.plot.width = 11, repr.plot.height = 8)
options(repr.matrix.max.rows = 6)

# Introduction

# Methods

In [18]:
#Read the data from the data folder
players <- read_csv("data/players.csv")|>
    as.data.frame()|>
    mutate(subscribe = as.factor(subscribe))
head(players) #used head as there are too many rows
#clean the data


Rows: 196 Columns: 7
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (4): experience, hashedEmail, name, gender
dbl (2): played_hours, Age
lgl (1): subscribe

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


,experience,subscribe,hashedEmail,played_hours,name,gender,Age
,<chr>,<fct>,<chr>,<dbl>,<chr>,<chr>,<dbl>
1,Pro,TRUE,f6daba428a5e19a3d47574858c13550499be23603422e6a0ee9728f8b53e192d,30.3,Morgan,Male,9
2,Veteran,TRUE,f3c813577c458ba0dfef80996f8f32c93b6e8af1fa939732842f2312358a88e9,3.8,Christian,Male,17
3,Veteran,FALSE,b674dd7ee0d24096d1c019615ce4d12b20fcbff12d79d3c5a9d2118eb7ccbb28,0.0,Blake,Male,17
4,Amateur,TRUE,23fe711e0e3b77f1da7aa221ab1192afe21648d47d2b4fa7a5a659ff443a0eb5,0.7,Flora,Female,21
5,Regular,TRUE,7dc01f10bf20671ecfccdac23812b1b415acd42c2147cb0af4d48fcce2420f3e,0.1,Kylie,Male,21
6,Amateur,TRUE,f58aad5996a435f16b0284a3b267f973f9af99e7a89bee0430055a44fa92f977,0.0,Adrian,Female,17


In [16]:
# session <- read_csv("data/session.csv")|>
#     as.data.frame()
# head(session)#not useful for this question

In order to find out which "kinds" of players are most likely to contribute a large amount of data, we need to make a model that predicts the played hours of the players.

Looking at the players data, there are four varibles that can be use to do the prediction : experience, subscribe, gender, and age. 